### IMPORT RELEVENT PACKAGES

In [62]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

### DATA

In [51]:
mnist_dataset, mnist_info = tfds.load(name='mnist',with_info=True, as_supervised=True)

In [52]:
mnist_train, mnist_test = mnist_dataset['train'],mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples,tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples,tf.int64)

def scale(image,label):
    image = tf.cast(image,tf.float32)
    image /= 255.
    return image, label
scaled_train_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

buffer_size = 10000  #SHUFFLE 7500 DATA @ ONCE
shuffled_train_validation_data = scaled_train_validation_data.shuffle(buffer_size)

In [53]:
validation_data = scaled_train_validation_data.take(num_validation_samples)
train_data = scaled_train_validation_data.skip(num_validation_samples)


In [54]:
batch_size = 100
train_data = train_data.batch(batch_size)
validation_data= validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

In [55]:
validation_inputs, validation_targets = next(iter(validation_data))

### NEURAL NET MODEL

In [68]:
input_size= 784
hidden_layer_size = 50
output_layer_size = 10
model = tf.keras.Sequential([
                                tf.keras.layers.Flatten(input_shape=(28,28,1)),
                                tf.keras.layers.Dense(hidden_layer_size,activation = 'relu'),
                                tf.keras.layers.Dense(hidden_layer_size,activation = 'relu'),
                                tf.keras.layers.Dense(output_layer_size,activation = 'softmax')
                            ])

In [69]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',metrics = ['accuracy'])

### TRAIN

In [70]:
num_epoch = 5
model.fit(train_data,epochs = num_epoch, validation_data = (validation_inputs, validation_targets),verbose = 2)

Epoch 1/5
540/540 - 3s - loss: 0.4048 - accuracy: 0.8830 - val_loss: 0.2158 - val_accuracy: 0.9350
Epoch 2/5
540/540 - 3s - loss: 0.1803 - accuracy: 0.9469 - val_loss: 0.1692 - val_accuracy: 0.9470
Epoch 3/5
540/540 - 3s - loss: 0.1392 - accuracy: 0.9588 - val_loss: 0.1503 - val_accuracy: 0.9535
Epoch 4/5
540/540 - 3s - loss: 0.1142 - accuracy: 0.9662 - val_loss: 0.1396 - val_accuracy: 0.9558
Epoch 5/5
540/540 - 3s - loss: 0.0969 - accuracy: 0.9719 - val_loss: 0.1319 - val_accuracy: 0.9593


In [72]:
test_loss, test_accuracy = model.evaluate(test_data)
print('Test_Loss: {0:.2f}. Test_Accuracy: {1:.2f}%'.format(test_loss,test_accuracy*100))

1/1 [==============================] - 0s 0s/step - loss: 0.1185 - accuracy: 0.9639
Test_Loss: 0.12. Test_Accuracy: 96.39%
